## Overview
* This is an Arabic licence plate recognition system
* 2 Main procedures were applied
  * Preprocessing on plate
  * Predicting the plate characters

# Import Libraries

In [ ]:
!pip install imutils
!pip install tensorflow
!pip install keras

In [ ]:
!unzip './cropped_letters.zip'  -d './cropped_letters'
!unzip './cropped_numbers.zip'  -d './cropped_numbers'
!unzip './test_images.zip'  -d './test_images'

### Including needed libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers as L
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize, rescale
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Dropout, Conv2D,BatchNormalization
from imutils import paths
from skimage.segmentation import clear_border
from PIL import Image

import argparse
import imutils
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import skimage 
#from imutils.perspective import four_point_transform
import os
import cv2
import csv
import glob
import skimage.io as io
import pickle
from skimage.color import rgb2gray, gray2rgb
%matplotlib inline

### Defining needed preprocessing functions.

In [4]:
# module level variables ##########################################################################
GAUSSIAN_SMOOTH_FILTER_SIZE = (5, 5)
ADAPTIVE_THRESH_BLOCK_SIZE = 19
ADAPTIVE_THRESH_WEIGHT = 9

###################################################################################################
def preprocess(imgOriginal):
    imgSaturation, imgGrayscale = extractValue(imgOriginal)

    imgMaxContrastGrayscale = maximizeContrast(imgGrayscale)

    imgBlurred = cv2.GaussianBlur(imgMaxContrastGrayscale, GAUSSIAN_SMOOTH_FILTER_SIZE, 0)

    imgThreshValue = cv2.threshold(imgBlurred, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    imgThreshSaturation = cv2.threshold(imgSaturation, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

    # plt.imshow(imgBlurred,cmap='gray')
    # plt.show()
    # plt.imshow(imgThreshValue,cmap='gray')
    # plt.show()

    # plt.imshow(imgSaturation,cmap='gray')
    # plt.show()
    # plt.imshow(imgThreshSaturation,cmap='gray')
    # plt.show()

    #imgThresh = cv2.adaptiveThreshold(imgBlurred, 255.0, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, ADAPTIVE_THRESH_BLOCK_SIZE, ADAPTIVE_THRESH_WEIGHT)
    
    return imgGrayscale, (imgThreshValue & (255 - imgThreshSaturation))
# end function

def preprocess2(imgOriginal):
    _, imgGrayscale = extractValue(imgOriginal)

    imgBlurred = cv2.GaussianBlur(imgGrayscale, GAUSSIAN_SMOOTH_FILTER_SIZE, 0)

    imgThreshValue = cv2.threshold(imgBlurred, 0, 255, cv2.THRESH_OTSU)[1]

    # plt.imshow(imgBlurred,cmap='gray')
    # plt.show()
    # plt.imshow(imgThreshValue,cmap='gray')
    # plt.show()

    # plt.imshow(imgSaturation,cmap='gray')
    # plt.show()
    # plt.imshow(imgThreshSaturation,cmap='gray')
    # plt.show()

    #imgThresh = cv2.adaptiveThreshold(imgBlurred, 255.0, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, ADAPTIVE_THRESH_BLOCK_SIZE, ADAPTIVE_THRESH_WEIGHT)
    
    return imgThreshValue
# end function

###################################################################################################
def extractValue(imgOriginal):
    height, width, numChannels = imgOriginal.shape

    imgHSV = np.zeros((height, width, 3), np.uint8)

    imgHSV = cv2.cvtColor(imgOriginal, cv2.COLOR_BGR2HSV)

    imgHue, imgSaturation, imgValue = cv2.split(imgHSV)

    return imgSaturation, imgValue
# end function

###################################################################################################
def maximizeContrast(imgGrayscale):

    height, width = imgGrayscale.shape

    imgTopHat = np.zeros((height, width, 1), np.uint8)
    imgBlackHat = np.zeros((height, width, 1), np.uint8)

    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    imgTopHat = cv2.morphologyEx(imgGrayscale, cv2.MORPH_TOPHAT, structuringElement)
    imgBlackHat = cv2.morphologyEx(imgGrayscale, cv2.MORPH_BLACKHAT, structuringElement)

    imgGrayscalePlusTopHat = cv2.add(imgGrayscale, imgTopHat)
    imgGrayscalePlusTopHatMinusBlackHat = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    return imgGrayscalePlusTopHatMinusBlackHat
# end function

In [5]:
def order_points(pts):
	# initialzie a list of coordinates that will be ordered
	# such that the first entry in the list is the top-left,
	# the second entry is the top-right, the third is the
	# bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")
	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]
	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]
	# return the ordered coordinates
	return rect

In [6]:
def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	(tl, tr, br, bl) = rect
	# compute the width of the new image, which will be the
	# maximum distance between bottom-right and bottom-left
	# x-coordiates or the top-right and top-left x-coordinates
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))
	# compute the height of the new image, which will be the
	# maximum distance between the top-right and bottom-right
	# y-coordinates or the top-left and bottom-left y-coordinates
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))
	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")
	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
	# return the warped image
	return warped

In [7]:
def contourApprox(imgOriginal, cnts):
	# initialize a contour that corresponds to the receipt outline
	receiptCnt = []
	firstContour = False
	firstArea = 0
	firstPeri = 0

	# loop over the contours
	for c in cnts:
		# approximate the contour
		peri = cv2.arcLength(c, True) + 0.01
		approx = cv2.approxPolyDP(c, 0.05 * peri, True)

		area = cv2.contourArea(approx) + 0.01

		drawnContour = imgOriginal.copy()
		cv2.drawContours(drawnContour, [approx], -1, (0, 255, 0), 2)
# 		plt.imshow(drawnContour)
# 		plt.show()
		
		if area < 3000:
			break

		# if our approximated contour has four points, then we can
		# assume we have found the outline of the receipt
		if len(approx) == 4 and (not firstContour or (firstPeri / peri < 1.4 and firstPeri / peri > 0.7 and firstArea / area < 1.4 and firstArea / area > 0.7)):
			receiptCnt.append(approx)
			if firstContour:
				 break
			firstContour = True
			firstArea = area
			firstPeri = peri
			
		else:
			approx = cv2.convexHull(approx)
			peri = cv2.arcLength(approx, True) + 0.01
			area = cv2.contourArea(approx) + 0.01
			if len(approx) == 4 and (not firstContour or (firstPeri / peri < 1.4 and firstPeri / peri > 0.7 and firstArea / area < 1.4 and firstArea / area > 0.7)):
				receiptCnt.append(approx)
				if firstContour:
					break
				firstContour = True
				firstArea = area
				firstPeri = peri
				break		
	# if the receipt contour is empty then our script could not find the
	# outline and we should be notified
	return receiptCnt

In [8]:
def showApprox(imgOriginal, thresh, receiptCnt):
    cv2.drawContours(imgOriginal, [receiptCnt], -1, (0, 255, 0), 2)
    #print(output[1])
    #plt.imshow(imgOriginal)
    #plt.show()
    rect_coor = np.array(receiptCnt.reshape(4,2))
    img = four_point_transform(imgOriginal, rect_coor)
    #print(img.shape)
    #img_new = add_margin(img, 5, 10, 0, 10)
    final_img = preprocess2(img)
#     plt.imshow(final_img, cmap="gray")
#     plt.show()
    return (final_img,img)
    #print(f"boundaries = {receiptCnt.reshape(4, 2)}")


In [9]:
def CutLetters(img,rgb_img, count):
    height, width = img.shape
    windowWidthR = int(0.18 * width * count)
    windowWidthL = int(0.12 * width * count)
    windowWidth = windowWidthR
    if count == 2:
        windowWidthR = windowWidthL
    startRatio = int(0.1 * windowWidth)
    windowHeight = img.shape[0]
    marginWidth = int(windowWidth * 0.05)
    stepSize = 8
    GroupSize = 8
    lowerBlackLimit = 0.07
    upperBlackLimit = 0.9

    Letters = []
    Letters_imgs = []
    Letters_imgs_thresh = []
    for i in range(0, width - (windowWidth), stepSize * GroupSize):

        Group = []
        for j in range(i, min(i + stepSize * GroupSize, width - (windowWidth)), stepSize):
            #plt.imshow(img[:, j:j+windowWidth],cmap='gray')
            #plt.show()
            if j <= width/2:
                windowWidth = windowWidthL
            else:
                windowWidth = windowWidthR

            blackCountMarginL = marginWidth * height  - np.count_nonzero(img[:, j : j+marginWidth])
            blackCountMarginR = marginWidth * height  - np.count_nonzero(img[:, j+windowWidth-marginWidth : j+windowWidth])
            blackCountInner = (windowWidth - 2 * startRatio) * height - np.count_nonzero(img[:, j+startRatio : j+windowWidth-startRatio])
            
            #print(blackCountMarginL, marginWidth * height)
            #print(blackCountMarginR, marginWidth * height)
            #print(blackCountInner, (windowWidth - 2 * marginWidth) * height)
            #and blackCountMarginR < 0.2 * marginWidth * height \

            if blackCountMarginL < 0.25 * marginWidth * height and blackCountMarginR < 0.17 * marginWidth * height and blackCountInner > lowerBlackLimit * (windowWidth - 2 * startRatio) * height \
                and blackCountInner < upperBlackLimit * (windowWidth - 2 * startRatio) * height:
                Group.append((blackCountInner, j))
        if len(Group) > 0:
            max_G = max(Group)[1]
            if len(Letters) == 0 or len(Letters) > 0 and max_G - Letters[-1] > 21:
                Letters.append(max_G)
              # rect_coord = [rgb_img[0][0],rgb_img[0][-1],max(Group)[1],max(Group)[1]+windowWidth]
              # rect_coord = np.array(rect_coord,dtype='int32')
                rgb2_img = rgb_img[:, max_G:max_G+windowWidth]
                Letters_imgs.append((rgb2_img, max_G))
                #               plt.imshow(rgb2_img)
                thresh_img = img[:, max_G:max_G+windowWidth]
                Letters_imgs_thresh.append((thresh_img, max_G))
              #plt.imshow(img[:, max(Group)[1]:max(Group)[1]+windowWidth],cmap='gray')
              #plt.show()

    return Letters_imgs,Letters_imgs_thresh






In [10]:
def sideBorder(img, d):
    h,w=img.shape[0:2]
    base_size=h+20,w+20,3
    if d == 1:
        base_size=h+20,w+20
    # make a 3 channel image for base which is slightly larger than target img
    base=np.zeros(base_size,dtype=np.uint8)
    if d == 1:
        cv2.rectangle(base,(0,0),(w+20,h+20),255,20) # really thick white rectangle
    else:
        cv2.rectangle(base,(0,0),(w+20,h+20),(255,255,255),20) # really thick white rectangle
    
    base[10:h+10,10:w+10]=img # this works
    return base

In [11]:
def ImagetoSymbols(imgOriginal):    
    #imgOriginal = cv2.imread("./train_images/02937.jpg")
    #img = imgOriginal.copy()
    imgOriginal = imutils.resize(imgOriginal, width=500)
    #ratio = img.shape[1] / float(imgOriginal.shape[1])
#     plt.imshow(imgOriginal)
#     plt.show()
    gray,thresh = preprocess(imgOriginal)
    # plt.imshow(gray,cmap='gray')
    # plt.show()
#     plt.imshow(thresh,cmap='gray')
#     plt.show()

    #ClosedThresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_RECT, (15, 3)))
    OpenedThresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7)))

    minValue = np.min(OpenedThresh)

    OpenedThresh[imgOriginal.shape[0]-4:imgOriginal.shape[0]-1, :] = minValue
    OpenedThresh[:, imgOriginal.shape[1]-4:imgOriginal.shape[1]-1] = minValue
    OpenedThresh[:, 0:3] = minValue


#     plt.imshow(OpenedThresh,cmap='gray')
#     plt.show()



    #plt.imshow(ClosedThresh,cmap='gray')
    #plt.show()
    

    cnts = cv2.findContours(OpenedThresh.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
    # for c in cnts:
        # #print(c[0])
        # plt.scatter(c[:, 0, 0], -c[:, 0, 1])
        # plt.show()
    # np.array(cnts,dtype=np.int32).shape
    #print(cnts[-1])
    receiptCnt = contourApprox(imgOriginal, cnts)
    if len(receiptCnt) == 0:
        
        receiptCnt = [np.array([[[0,  0]], [[0, imgOriginal.shape[0]]],
         [[imgOriginal.shape[1], imgOriginal.shape[0]]], [[imgOriginal.shape[1], 0]]])]
    #print(receiptCnt)

    numbers_cropped = []
    numbers_cropped_thresh = []
    letters_cropped = []
    letters_cropped_thresh = []


    if len(receiptCnt) == 2:
        x = 1
        avg1 = np.average(receiptCnt[0][:,0,0])
        avg2 = np.average(receiptCnt[1][:,0,0])
        if avg2 < avg1:
            receiptCnt[0],receiptCnt[1] = receiptCnt[1],receiptCnt[0] 

        final_img1 ,rgb_img1 = showApprox(imgOriginal, thresh, receiptCnt[0])
        final_img1 = sideBorder(final_img1, 1)
        rgb_img1 = sideBorder(rgb_img1, 3)



        symbols_cropped, symbols_cropped_thresh = CutLetters(final_img1,rgb_img1, 2)
        for i in range(len(symbols_cropped)):
            numbers_cropped.append(symbols_cropped[i][0])
            numbers_cropped_thresh.append(symbols_cropped_thresh[i][0])

        final_img1 ,rgb_img1 = showApprox(imgOriginal, thresh, receiptCnt[1])
        final_img1 = sideBorder(final_img1, 1)
        rgb_img1 = sideBorder(rgb_img1, 3)



        symbols_cropped, symbols_cropped_thresh = CutLetters(final_img1,rgb_img1, 2)
        for i in range(len(symbols_cropped)):
            letters_cropped.append(symbols_cropped[i][0])
            letters_cropped_thresh.append(symbols_cropped_thresh[i][0])

    
    else:
        avg = np.average(receiptCnt[0][:,0,0])
        final_img1 ,rgb_img1 = showApprox(imgOriginal, thresh, receiptCnt[0])
        final_img1 = sideBorder(final_img1, 1)
        rgb_img1 = sideBorder(rgb_img1, 3)

        if final_img1.shape[1] < 280:
            if avg < final_img1.shape[1] / 2:
                symbols_cropped, symbols_cropped_thresh = CutLetters(final_img1,rgb_img1, 2)
                for i in range(len(symbols_cropped)):
                    numbers_cropped.append(symbols_cropped[i][0])
                    numbers_cropped_thresh.append(symbols_cropped_thresh[i][0])
            else:
                symbols_cropped, symbols_cropped_thresh = CutLetters(final_img1,rgb_img1, 2)
                for i in range(len(symbols_cropped)):
                    letters_cropped.append(symbols_cropped[i][0])
                    letters_cropped_thresh.append(symbols_cropped_thresh[i][0])
        else: 
            symbols_cropped, symbols_cropped_thresh = CutLetters(final_img1,rgb_img1, 1)
            for i in range(len(symbols_cropped)):
                if symbols_cropped[i][1] < final_img1.shape[1] / 2:
                    numbers_cropped.append(symbols_cropped[i][0])
                    numbers_cropped_thresh.append(symbols_cropped_thresh[i][0])
                else:
                    letters_cropped.append(symbols_cropped[i][0])
                    letters_cropped_thresh.append(symbols_cropped_thresh[i][0])
    return numbers_cropped_thresh, letters_cropped_thresh
    #print(numbers_cropped_thresh)
    #for r in range(receiptCnt):


    #    final_img ,rgb_img= showApprox(imgOriginal, thresh, r)
    #    final_img = sideBorder(final_img, 1)
    #    rgb_img = sideBorder(rgb_img, 3)


    #    letters_cropped, letters_cropped_thresh = CutLetters(final_img,rgb_img, len(receiptCnt))
    #    for character in letters_cropped_thresh:
    #        charcater = np.array(character)
    #        plt.imshow(charcater)
    #        plt.show()
    #        #print(predict(img=character, model=model, labels=labels))
    #        print(predict(character))
    #      #print(final_img.shape[1])
    #      #CharacterContours(final_img)

# plt.plot(cnts[0])
# plt.show()

#cnts = skimage.measure.find_contours(thresh)
#cnts = sorted(cnts, reverse=True)
#cnts

# for c in cnts:
#     # compute the bounding box of the contour and then use
#     # the bounding box to derive the aspect ratio
#     (x, y, w, h) = cv2.boundingRect(c)
#     ar = w / float(h)
#     if ar >= 2 and ar <= 10:
#         lpCnt = c
#         letter = gray[y:y + h, x:x + w]
#         roi = cv2.threshold(letter, 0, 255,
#                 cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
#             # check to see if we should clear any foreground
#             # pixels touching the border of the image
#             # (which typically, not but always, indicates noise)
#         # display any debugging information and then break
#         # from the loop early since we have found the license
#         # plate region
#         # plt.plot(letter,cmap='gray')
#         # plt.show()
#         plt.plot(roi,cmap='gray')
#         plt.show()
#         print("****************")
# print(roi)   

In [12]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

def change_size(image):
    img = array_to_img(image, scale=False) #returns PIL Image
    img = img.resize((32, 32)) #resize image
    arr = img_to_array(img) #convert back to array
    return arr.astype(np.float64)

In [13]:
trainData = []
trainLabels = []

letter_directory="./cropped_letters/cropped_letters/*/*"
number_directory="./cropped_numbers/cropped_numbers/*/*"
paths = glob.glob(letter_directory)
paths = paths + glob.glob(number_directory)

for path in paths:
    img = io.imread(path)
    imgSaturation, imgGrayscale = extractValue(img)
    #imgMaxContrastGrayscale = maximizeContrast(imgGrayscale)
    #imgBlurred = cv2.GaussianBlur(imgGrayscale, GAUSSIAN_SMOOTH_FILTER_SIZE, 0)
    #imgThreshValue = cv2.threshold(imgBlurred, 0, 255, cv2.THRESH_OTSU)[1]
#     imgPadded = gray2rgb(imgThreshValue)
    imgPadded = resize(img,(32,32,1) )
    trainData.append(imgPadded)
    img_label = path.split('/')[-2]
    trainLabels.append(img_label)

trainData = np.array(trainData)
trainLabels = np.array(trainLabels)

print(trainLabels.shape)
print(trainData.shape)
print(trainData[0].shape)

(25002,)
(25002, 32, 32, 1)
(32, 32, 1)


## Apply augmentation
- rotation, shifts, brightness change

In [14]:
# The flow_from_directory() method allows you to read the images directly from the directory and augment them while the neural network model is learning on the training data.
train_datagen = ImageDataGenerator(rescale=1./255,)
#     shear_range=0.1,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
# #     zoom_range=0.2,
#     rotation_range=20,
    # validation_split=0.2) # set validation split

# train_generator = train_datagen.flow(
#     x=trainData, 
#     y=trainLabels,
#     batch_size=128) # set as training data
train_generator = train_datagen.flow_from_directory(
        './cropped_letters/cropped_letters',  # This is the source directory for training images
        target_size=(32, 32),
        batch_size=128,
        class_mode='categorical')

#     validation_generator = train_datagen.flow(
#     x=trainData, 
#     y=trainLabels,
#     batch_size=20,
#     subset='validation') # set as validation data

Found 12509 images belonging to 28 classes.


We split the data into the train and validation sets. Here is the distribution of the split data.

In [ ]:
# img, label = train_generator.next()
# randindx = random.randint(0,128)
# plt.imshow(img[randindx])

In [15]:
letter_model = Sequential([
    tf.keras.layers.Conv2D(16, (22,22), input_shape=(32, 32, 3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(16, (22,22), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, (16,16), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(32, (16,16), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, (8,8), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(64, (4,4), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(128, (4,4), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(128, (4,4), activation='relu', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(28, activation='softmax')
])

letter_model.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
history = letter_model.fit(train_generator, epochs=18, 
            steps_per_epoch=train_generator.n//train_generator.batch_size)

Epoch 1/18


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


97/97 [==============================] - 12s 78ms/step - loss: 2.7076 - accuracy: 0.1838
Epoch 2/18
97/97 [==============================] - 9s 89ms/step - loss: 1.0012 - accuracy: 0.6324
Epoch 3/18
97/97 [==============================] - 8s 86ms/step - loss: 0.5509 - accuracy: 0.7973
Epoch 4/18
97/97 [==============================] - 5s 49ms/step - loss: 0.3770 - accuracy: 0.8671
Epoch 5/18
97/97 [==============================] - 5s 48ms/step - loss: 0.2955 - accuracy: 0.8956
Epoch 6/18
97/97 [==============================] - 5s 48ms/step - loss: 0.2504 - accuracy: 0.9146
Epoch 7/18
97/97 [==============================] - 5s 48ms/step - loss: 0.2108 - accuracy: 0.9257
Epoch 8/18
97/97 [==============================] - 5s 50ms/step - loss: 0.1761 - accuracy: 0.9383
Epoch 9/18
97/97 [==============================] - 5s 49ms/step - loss: 0.1486 - accuracy: 0.9480
Epoch 10/18
97/97 [==============================] - 5s 50ms/step - loss: 0.1395 - accuracy: 0.9521
Epoch 11/18
97/97 [

In [16]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        
        if(logs.get('accuracy') > 0.98):
            # Stop if threshold is met
            print("\naccuracy of training is bigger than 98!")
            self.model.stop_training = True

# Instantiate class
callbacks = myCallback()

In [17]:
train_generator2 = train_datagen.flow_from_directory(
        './cropped_numbers/cropped_numbers',  # This is the source directory for training images
        target_size=(32, 32),  
        batch_size=128,
        class_mode='categorical')

Found 12493 images belonging to 10 classes.


In [18]:
number_model = Sequential([
  tf.keras.layers.Conv2D(16, (22,22), input_shape=(32, 32, 3), activation='relu', padding='same'),
  tf.keras.layers.Conv2D(16, (22,22), activation='relu', padding='same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D((2,2)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(32, (16,16), activation='relu', padding='same'),
  tf.keras.layers.Conv2D(32, (16,16), activation='relu', padding='same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D((2,2)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(64, (8,8), activation='relu', padding='same'),
  tf.keras.layers.Conv2D(64, (4,4), activation='relu', padding='same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D((2,2)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv2D(128, (4,4), activation='relu', padding='same'),
  tf.keras.layers.Conv2D(128, (4,4), activation='relu', padding='same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.2),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

number_model.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
history = number_model.fit(train_generator2, epochs=8, callbacks=callbacks)

Epoch 1/8


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


98/98 [==============================] - 7s 59ms/step - loss: 2.1499 - accuracy: 0.2081
Epoch 2/8
98/98 [==============================] - 5s 56ms/step - loss: 0.5876 - accuracy: 0.8061
Epoch 3/8
98/98 [==============================] - 5s 48ms/step - loss: 0.1795 - accuracy: 0.9485
Epoch 4/8
98/98 [==============================] - 5s 49ms/step - loss: 0.0918 - accuracy: 0.9743
Epoch 5/8
97/98 [============================>.] - ETA: 0s - loss: 0.0696 - accuracy: 0.9803
accuracy of training is bigger than 98!
98/98 [==============================] - 5s 50ms/step - loss: 0.0697 - accuracy: 0.9802


In [19]:
number_labels = {v: k for k, v in train_generator2.class_indices.items()}
letter_labels = {v: k for k, v in train_generator.class_indices.items()}
print(number_labels)
print(letter_labels)

{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9'}
{0: 'ا', 1: 'ب', 2: 'ت', 3: 'ث', 4: 'ج', 5: 'ح', 6: 'خ', 7: 'د', 8: 'ذ', 9: 'ر', 10: 'ز', 11: 'س', 12: 'ش', 13: 'ص', 14: 'ض', 15: 'ط', 16: 'ظ', 17: 'ع', 18: 'غ', 19: 'ف', 20: 'ق', 21: 'ك', 22: 'ل', 23: 'م', 24: 'ن', 25: 'ه', 26: 'و', 27: 'ى'}


In [20]:
def predict(img, model, labels):
    img = resize(img, (32,32,3), anti_aliasing=True)
    img = np.expand_dims(img, axis=0)

    preds = model.predict(img)
    return labels[np.argmax(preds)]

In [40]:
import csv
rows = []
for i in range(691):
    imgOriginal = io.imread(f"./test_images/test_images/{str(i).zfill(5)}.jpg") # to be replaced by image readerfrom loop
    Nums, Chars = ImagetoSymbols(imgOriginal)

    labelList = []

    for char in Chars:
      # predicting with letters' model
      predicted_label = predict(char, model=letter_model, labels=letter_labels)
      labelList.append(predicted_label)  
      # plt.imshow(np.array(char))
      # plt.show()
      # print(predicted_label)

    for num in Nums:
      # predicting with numbers' model
      predicted_label = predict(num, model=number_model, labels=number_labels)
      labelList.append(predicted_label) 
      # plt.imshow(np.array(num))
      # plt.show()
      # print(predicted_label)
      
    label = ''.join(str(e) for e in labelList) 

    # csv header
    headerNames = ['img_name', 'label']

    # csv data
    imageID = f'{str(i).zfill(5)}.jpg' #replace with img id from loop
    rows.append( 
        {'img_name': imageID,
        'label': label})

with open('./submissions.csv', 'w', encoding='UTF16', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=headerNames)
    writer.writeheader()
    writer.writerows(rows)

In [44]:
df = pd.read_csv("./submissions.csv", encoding='UTF16')
df.columns

Index(['img_name', 'label'], dtype='object')